## Пишем классификатор: попытка №1

In [1]:
from nltk.corpus import stopwords
import csv
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.metrics import confusion_matrix
import os
from langdetect import detect
import re
import pandas as pd
import langid

Объединим датасет с удаленными и добавим столбец check, который и будет целевым признаком для модели. 0 - хорошая аннотация, 1 - требует удаления

In [2]:
folder_path = 'openalex'
df_combined = pd.DataFrame() 
# Проходим по всем файлам CSV в папке и объединяем их
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            df_combined = pd.concat([df_combined, df], ignore_index=True)


df_combined.to_csv('combined_file.csv', index=False)

In [3]:
df1 = pd.read_csv('combined_file.csv')
df1['check'] = 0
print(len(df1))
df2 = pd.read_csv('output_del2.csv')
df2['check'] = 1
print(len(df2))
df_res = pd.concat([df1, df2], ignore_index=True)
print(len(df_res))
df_res.to_csv('df_res.csv', index = False)
# Удаление дубликатов строк с одинаковыми значениями в столбце 'id', оставляя только строки с 'check' равным 1
df_res2 = df_res.drop_duplicates(subset='id', keep='last')
print(len(df_res2))


/tmp/ipykernel_226/3195477132.py:1: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('combined_file.csv')


259595
130694
390289
254579


In [7]:
df_res2.to_csv('df_res2.csv', index = False)

Поделим на train и test, уберем лишние признаки

In [8]:
train, test = train_test_split(df_res2, test_size=0.2)
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [9]:
train_df = pd.read_csv('train.csv')
train_df = train_df[['id', 'abstract', 'check']]
train_df.to_csv('train_updated.csv', index=False)
test_df = pd.read_csv('test.csv')
test_df = test_df[['id', 'abstract', 'check']]
test_df.to_csv('test_updated.csv', index=False)

In [10]:
df_res2 = df_res2[['id', 'abstract', 'check']]

In [11]:
df_res2['abstract'] = df_res2['abstract'].fillna('')

In [12]:
data = df_res2["abstract"]
y = df_res2["check"]
Counter(y)

Counter({0: 123903, 1: 130676})

In [13]:
data_train, data_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=0)

#### Попробуем векторизировать через CountVectorizer и предсказать через модель логистической регрессии

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
cvec = CountVectorizer(binary=True, max_features=10000)
cvec.fit(data_train)

CountVectorizer(binary=True, max_features=10000)

In [16]:
X_train = cvec.transform(data_train).astype(int)
X_test = cvec.transform(data_test).astype(int)

In [17]:
feature_names = cvec.get_feature_names()
feature_names[-20:]

/home/anna_mikhneva_/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['été',
 'évidence',
 'évolution',
 'être',
 'über',
 'œuvre',
 'še',
 'że',
 'для',
 'из',
 'или',
 'как',
 'на',
 'не',
 'от',
 'по',
 'статье',
 'что',
 'языка',
 'языке']

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
model = LogisticRegression()
model.fit(X_train, y_train)

/home/anna_mikhneva_/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [20]:
from sklearn.metrics import accuracy_score, f1_score

In [21]:
model.predict(X_test)[:5]

array([0, 1, 1, 1, 0])

In [22]:
model.predict_proba(X_test)[:5].round(2)

array([[0.99, 0.01],
       [0.24, 0.76],
       [0.24, 0.76],
       [0.24, 0.76],
       [1.  , 0.  ]])

In [23]:
y_pred = model.predict(X_test)

In [24]:
accuracy_score(y_test, y_pred)

0.8291892528871082

In [25]:
f1_score(y_test, y_pred)

0.848153644696639

Строим матрицу ошибок и создаем файлы с false positive и false negative

In [26]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[17930  6867]
 [ 1830 24289]]


In [46]:
false_positive_indices = data_test.index[(y_pred == 1) & (y_test == 0)] #не удалил, модель сказала удалить
false_negative_indices = data_test.index[(y_pred == 0) & (y_test == 1)] #удалил, а модель не удалила
false_positive_examples = data_test.loc[false_positive_indices]
false_negative_examples = data_test.loc[false_negative_indices]
print("False Positive Examples:")
print(false_positive_examples[:50])
print("\nFalse Negative Examples:")
print(false_negative_examples[:50])

False Positive Examples:
190402                                                     
97846                                                      
1620                                                       
171269                                                     
212005                                                     
245806                                                     
98430     In 1968, I spent the months of May and June co...
64619     Sarama Babiat Review Of The Dance Form. The pu...
74618                         Abstract The pragmatic marker
66769                                                      
252118                                                     
97690                                                      
217954    L'A. developpe quelques arguments qui tendent ...
146055                                                     
158670                                                     
192669                                                     
124576         

In [52]:
false_positive_df = pd.DataFrame(columns=[ 'abstract', 'pred', 'true'])
false_negative_df = pd.DataFrame(columns=[ 'abstract', 'pred', 'true'])

for example in false_positive_examples:
    false_positive_df = false_positive_df.append({ 'abstract': example, 'pred': 1, 'true': 0}, ignore_index=True)

/tmp/ipykernel_176/743083802.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  false_positive_df = false_positive_df.append({ 'abstract': example, 'pred': 1, 'true': 0}, ignore_index=True)
/tmp/ipykernel_176/743083802.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  false_positive_df = false_positive_df.append({ 'abstract': example, 'pred': 1, 'true': 0}, ignore_index=True)
/tmp/ipykernel_176/743083802.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  false_positive_df = false_positive_df.append({ 'abstract': example, 'pred': 1, 'true': 0}, ignore_index=True)
/tmp/ipykernel_176/743083802.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

In [53]:
for example in false_negative_examples:
    false_negative_df = false_negative_df.append({ 'abstract': example, 'pred': 0, 'true': 1}, ignore_index=True)

/tmp/ipykernel_176/967258037.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  false_negative_df = false_negative_df.append({ 'abstract': example, 'pred': 0, 'true': 1}, ignore_index=True)
/tmp/ipykernel_176/967258037.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  false_negative_df = false_negative_df.append({ 'abstract': example, 'pred': 0, 'true': 1}, ignore_index=True)
/tmp/ipykernel_176/967258037.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  false_negative_df = false_negative_df.append({ 'abstract': example, 'pred': 0, 'true': 1}, ignore_index=True)
/tmp/ipykernel_176/967258037.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

In [54]:
false_positive_df.to_csv('false_positive_df.csv', index=False)
false_negative_df.to_csv('false_negative_df.csv', index=False)

Проверим гипотезу о том, изменится ли что-то, если подготовить текстовые данные перед векторизацией

In [54]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [55]:
def clean_text(text):
    for i in range(len(text) - 1):
        if text[i] in ".,?!;:":
            if(i != len(text) - 1):
                text = text[:i] + " " + text[i+1:]
            else:
                text = text[:i-1]
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [56]:
df_res2["abstract"] = df_res2["abstract"].apply(lambda x: clean_text(x))

In [57]:
data = df_res2["abstract"]
y = df_res2["check"]
Counter(y)

Counter({0: 123903, 1: 130676})

In [58]:
data_train, data_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=0)
cvec = CountVectorizer(binary=True, max_features=10000)
cvec.fit(data_train)
X_train = cvec.transform(data_train).astype(int)
X_test = cvec.transform(data_test).astype(int)

In [59]:
model = LogisticRegression()
model.fit(X_train, y_train)

/home/anna_mikhneva_/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [60]:
y_pred = model.predict(X_test)

In [62]:
print(accuracy_score(y_test, y_pred))
f1_score(y_test, y_pred)

0.8217259800455653


0.8428850847281601

Просмотрев файлы false_positive, было замечено много таких, которые были пропущены при удалении вручную, поэтому для чистоты модели все пустые аннотации и аннотации, на отличном от английского языках были переотмечены как необходимые к удалению (check == 1)

In [ ]:
df = pd.read_csv('df_res2.csv')

# Создание списка для записи строк, которые нужно удалить
rows_to_delete = []

# Проверка каждой строки
for index, row in df.iterrows():
    if pd.isnull(row['abstract']) or row['abstract'] == 'NA' or row['abstract'] == '':
        if row['check'] == 0:
            rows_to_delete.append(row)
            df.at[index, 'check'] = 1
    elif langid.classify(row['abstract'])[0] != 'en' and row['check'] == 0:
        rows_to_delete.append(row)
        df.at[index, 'check'] = 1

# Создание DataFrame из списка строк для удаления
df_delete = pd.concat(rows_to_delete, axis=1).T

# Сохранение измененной исходной таблицы
df.to_csv('df_res2_modified.csv', index=False)

# Сохранение таблицы для удаления
df_delete.to_csv('should_delete.csv', index=False)